# 🐼 Pandas vs 🐻‍❄️ Polars: Análisis Comparativo

**Objetivo**: Comparar rendimiento, memoria y características de ambas librerías.

**Entorno**: Local con recursos dedicados (resultados consistentes y reproducibles).

---
## 1. Configuración Inicial

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import time
import gc
import os
import platform
import psutil
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

# Estilo de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Colores consistentes
PANDAS_COLOR = '#1E88E5'
POLARS_COLOR = '#43A047'

print("✅ Librerías cargadas")
print(f"📦 Pandas: {pd.__version__}")
print(f"📦 Polars: {pl.__version__}")

In [ ]:
# Información del sistema
def get_system_info():
    cpu_freq = psutil.cpu_freq()
    memory = psutil.virtual_memory()
    
    info = {
        'OS': f"{platform.system()} {platform.release()}",
        'Python': platform.python_version(),
        'CPU': platform.processor() or 'Unknown',
        'Cores': os.cpu_count(),
        'RAM': f"{memory.total / (1024**3):.1f} GB",
    }
    return info

sys_info = get_system_info()
print("\n📋 Sistema:")
for k, v in sys_info.items():
    print(f"   {k}: {v}")

In [ ]:
# Utilidades
class DataGenerator:
    """Genera datos sintéticos de ventas"""
    
    @staticmethod
    def create(n_rows: int, seed: int = 42) -> dict:
        np.random.seed(seed)
        
        return {
            'order_id': np.arange(1, n_rows + 1),
            'date': pd.date_range('2020-01-01', periods=n_rows, freq='1min'),
            'customer_id': np.random.randint(1, n_rows // 10 + 1, n_rows),
            'product_id': np.random.randint(1, 10000, n_rows),
            'category': np.random.choice(
                ['Electronics', 'Clothing', 'Food', 'Home', 'Sports'], n_rows
            ),
            'region': np.random.choice(
                ['North', 'South', 'East', 'West', 'Central'], n_rows
            ),
            'quantity': np.random.randint(1, 20, n_rows),
            'unit_price': np.round(np.random.uniform(5, 500, n_rows), 2),
            'discount': np.round(np.random.uniform(0, 0.3, n_rows), 2),
            'is_returned': np.random.choice([True, False], n_rows, p=[0.05, 0.95])
        }

def benchmark(func, warmup: int = 2, runs: int = 3):
    """Benchmark simple con warmup"""
    # Warmup
    for _ in range(warmup):
        func()
        gc.collect()
    
    # Medición
    times = []
    for _ in range(runs):
        gc.collect()
        start = time.perf_counter()
        result = func()
        times.append(time.perf_counter() - start)
    
    return {'median': np.median(times), 'times': times, 'result': result}

print("✅ Utilidades cargadas")

---
## 2. Crear Dataset de Prueba

In [ ]:
# Dataset principal: 1 millón de filas
N_ROWS = 1_000_000

print(f"📊 Generando {N_ROWS:,} filas...")
raw_data = DataGenerator.create(N_ROWS)

# Crear DataFrames
df_pandas = pd.DataFrame(raw_data)
df_polars = pl.DataFrame(raw_data)

print(f"✅ DataFrames creados")
print(f"\n📋 Columnas: {list(df_pandas.columns)}")
print(f"📋 Shape: {df_pandas.shape}")

df_pandas.head()

---
## 3. Test 1: Rendimiento en Operaciones Comunes

In [ ]:
print("="*60)
print("TEST 1: RENDIMIENTO EN OPERACIONES COMUNES")
print("="*60)

results = []

# 1.1 Filtrado complejo
print("\n🔍 1.1 Filtrado complejo...")

r_pd = benchmark(lambda: df_pandas[
    (df_pandas['category'].isin(['Electronics', 'Clothing'])) &
    (df_pandas['quantity'] > 5) &
    (df_pandas['unit_price'] > 100)
])

r_pl = benchmark(lambda: df_polars.filter(
    (pl.col('category').is_in(['Electronics', 'Clothing'])) &
    (pl.col('quantity') > 5) &
    (pl.col('unit_price') > 100)
))

results.append({'Operación': 'Filtrado complejo', 'Pandas': r_pd['median'], 'Polars': r_pl['median']})
print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {r_pd['median']/r_pl['median']:.2f}x")

In [ ]:
# 1.2 GroupBy + Agregaciones
print("\n📊 1.2 GroupBy + Agregaciones múltiples...")

r_pd = benchmark(lambda: df_pandas.groupby(['category', 'region']).agg({
    'quantity': ['sum', 'mean'],
    'unit_price': ['mean', 'min', 'max'],
    'order_id': 'count'
}))

r_pl = benchmark(lambda: df_polars.group_by(['category', 'region']).agg([
    pl.col('quantity').sum().alias('qty_sum'),
    pl.col('quantity').mean().alias('qty_mean'),
    pl.col('unit_price').mean().alias('price_mean'),
    pl.col('unit_price').min().alias('price_min'),
    pl.col('unit_price').max().alias('price_max'),
    pl.col('order_id').count().alias('count')
]))

results.append({'Operación': 'GroupBy + Agregaciones', 'Pandas': r_pd['median'], 'Polars': r_pl['median']})
print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {r_pd['median']/r_pl['median']:.2f}x")

In [ ]:
# 1.3 Ordenamiento
print("\n🔢 1.3 Ordenamiento múltiple...")

r_pd = benchmark(lambda: df_pandas.sort_values(
    ['category', 'unit_price', 'quantity'], 
    ascending=[True, False, True]
))

r_pl = benchmark(lambda: df_polars.sort(
    ['category', 'unit_price', 'quantity'],
    descending=[False, True, False]
))

results.append({'Operación': 'Ordenamiento', 'Pandas': r_pd['median'], 'Polars': r_pl['median']})
print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {r_pd['median']/r_pl['median']:.2f}x")

In [ ]:
# 1.4 Window Functions
print("\n🪟 1.4 Window Functions...")

r_pd = benchmark(lambda: df_pandas.assign(
    running_total=df_pandas.groupby('customer_id')['quantity'].cumsum(),
    price_rank=df_pandas.groupby('category')['unit_price'].rank(method='dense')
))

r_pl = benchmark(lambda: df_polars.with_columns([
    pl.col('quantity').cum_sum().over('customer_id').alias('running_total'),
    pl.col('unit_price').rank(method='dense').over('category').alias('price_rank')
]))

results.append({'Operación': 'Window Functions', 'Pandas': r_pd['median'], 'Polars': r_pl['median']})
print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {r_pd['median']/r_pl['median']:.2f}x")

In [ ]:
# 1.5 Crear columnas calculadas
print("\n➕ 1.5 Columnas calculadas...")

r_pd = benchmark(lambda: df_pandas.assign(
    revenue=df_pandas['quantity'] * df_pandas['unit_price'] * (1 - df_pandas['discount']),
    price_category=pd.cut(df_pandas['unit_price'], bins=[0, 50, 150, 500], labels=['Low', 'Medium', 'High'])
))

r_pl = benchmark(lambda: df_polars.with_columns([
    (pl.col('quantity') * pl.col('unit_price') * (1 - pl.col('discount'))).alias('revenue'),
    pl.when(pl.col('unit_price') <= 50).then(pl.lit('Low'))
      .when(pl.col('unit_price') <= 150).then(pl.lit('Medium'))
      .otherwise(pl.lit('High')).alias('price_category')
]))

results.append({'Operación': 'Columnas calculadas', 'Pandas': r_pd['median'], 'Polars': r_pl['median']})
print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {r_pd['median']/r_pl['median']:.2f}x")

In [ ]:
# Resumen Test 1
df_results = pd.DataFrame(results)
df_results['Speedup'] = df_results['Pandas'] / df_results['Polars']

print("\n" + "="*60)
print("📊 RESUMEN TEST 1: RENDIMIENTO")
print("="*60)

# Tabla
display(df_results.style.format({
    'Pandas': '{:.4f}s',
    'Polars': '{:.4f}s', 
    'Speedup': '{:.2f}x'
}).background_gradient(subset=['Speedup'], cmap='Greens'))

# Gráfico
fig, ax = plt.subplots(figsize=(12, 5))

x = np.arange(len(df_results))
width = 0.35

bars1 = ax.bar(x - width/2, df_results['Pandas']*1000, width, label='Pandas', color=PANDAS_COLOR, alpha=0.8)
bars2 = ax.bar(x + width/2, df_results['Polars']*1000, width, label='Polars', color=POLARS_COLOR, alpha=0.8)

ax.set_ylabel('Tiempo (ms)', fontweight='bold')
ax.set_title('⏱️ Tiempo de Ejecución por Operación', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(df_results['Operación'], rotation=15, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Añadir speedup sobre las barras
for i, speedup in enumerate(df_results['Speedup']):
    ax.annotate(f'{speedup:.1f}x', xy=(i, max(df_results['Pandas'].iloc[i], df_results['Polars'].iloc[i])*1000),
                xytext=(0, 10), textcoords='offset points', ha='center', fontweight='bold', color=POLARS_COLOR)

plt.tight_layout()
plt.show()

print(f"\n⚡ Speedup promedio: {df_results['Speedup'].mean():.2f}x")

---
## 4. Test 2: Modelo de Ejecución (Lazy vs Eager)

In [ ]:
print("="*60)
print("TEST 2: MODELO DE EJECUCIÓN")
print("="*60)

print("""
┌─────────────────────────────────────────────────────────────┐
│ PANDAS: Evaluación EAGER (inmediata)                        │
│ • Cada operación se ejecuta al momento                      │
│ • Resultados intermedios se materializan                    │
│ • Simple pero puede ser ineficiente                         │
├─────────────────────────────────────────────────────────────┤
│ POLARS: Evaluación LAZY (diferida) disponible               │
│ • Construye un plan de ejecución                            │
│ • Optimiza automáticamente (predicate pushdown, etc.)       │
│ • Ejecuta solo al llamar .collect()                         │
└─────────────────────────────────────────────────────────────┘
""")

In [ ]:
# Demostración de Lazy Evaluation
print("🦥 Lazy Evaluation en Polars:")
print("─" * 50)

# Crear query lazy (no ejecuta nada)
lazy_query = (
    df_polars.lazy()
    .filter(pl.col('category') == 'Electronics')
    .with_columns([
        (pl.col('quantity') * pl.col('unit_price') * (1 - pl.col('discount'))).alias('revenue')
    ])
    .group_by('region')
    .agg([
        pl.col('revenue').sum().alias('total_revenue'),
        pl.col('order_id').count().alias('orders')
    ])
    .sort('total_revenue', descending=True)
)

print("\n📋 Plan de ejecución optimizado:")
print(lazy_query.explain())

In [ ]:
# Comparar Lazy vs Eager
print("\n⏱️ Comparación Lazy vs Eager:")
print("─" * 50)

# Pandas (siempre eager)
r_pandas = benchmark(lambda: (
    df_pandas[df_pandas['category'] == 'Electronics']
    .assign(revenue=lambda x: x['quantity'] * x['unit_price'] * (1 - x['discount']))
    .groupby('region')
    .agg({'revenue': 'sum', 'order_id': 'count'})
    .rename(columns={'revenue': 'total_revenue', 'order_id': 'orders'})
    .sort_values('total_revenue', ascending=False)
))

# Polars Eager
r_polars_eager = benchmark(lambda: (
    df_polars
    .filter(pl.col('category') == 'Electronics')
    .with_columns([(pl.col('quantity') * pl.col('unit_price') * (1 - pl.col('discount'))).alias('revenue')])
    .group_by('region')
    .agg([pl.col('revenue').sum().alias('total_revenue'), pl.col('order_id').count().alias('orders')])
    .sort('total_revenue', descending=True)
))

# Polars Lazy
r_polars_lazy = benchmark(lambda: (
    df_polars.lazy()
    .filter(pl.col('category') == 'Electronics')
    .with_columns([(pl.col('quantity') * pl.col('unit_price') * (1 - pl.col('discount'))).alias('revenue')])
    .group_by('region')
    .agg([pl.col('revenue').sum().alias('total_revenue'), pl.col('order_id').count().alias('orders')])
    .sort('total_revenue', descending=True)
    .collect()
))

print(f"Pandas (Eager):       {r_pandas['median']*1000:>8.2f} ms")
print(f"Polars (Eager):       {r_polars_eager['median']*1000:>8.2f} ms  ({r_pandas['median']/r_polars_eager['median']:.2f}x vs Pandas)")
print(f"Polars (Lazy):        {r_polars_lazy['median']*1000:>8.2f} ms  ({r_pandas['median']/r_polars_lazy['median']:.2f}x vs Pandas)")

---
## 5. Test 3: Consumo de Memoria

In [ ]:
print("="*60)
print("TEST 3: CONSUMO DE MEMORIA")
print("="*60)

# Memoria base
mem_pandas = df_pandas.memory_usage(deep=True).sum() / 1024**2
mem_polars = df_polars.estimated_size() / 1024**2

print(f"\n📊 Memoria del DataFrame ({N_ROWS:,} filas):")
print(f"   Pandas: {mem_pandas:.2f} MB")
print(f"   Polars: {mem_polars:.2f} MB")
print(f"   💾 Ahorro: {((mem_pandas - mem_polars) / mem_pandas) * 100:.1f}%")

# Visualización
fig, ax = plt.subplots(figsize=(8, 5))

bars = ax.bar(['Pandas', 'Polars'], [mem_pandas, mem_polars], 
              color=[PANDAS_COLOR, POLARS_COLOR], alpha=0.8, edgecolor='white', linewidth=2)

for bar, val in zip(bars, [mem_pandas, mem_polars]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{val:.1f} MB', ha='center', fontweight='bold', fontsize=12)

ax.set_ylabel('Memoria (MB)', fontweight='bold')
ax.set_title(f'💾 Uso de Memoria ({N_ROWS:,} filas)', fontweight='bold', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Tipos de datos
print("\n📋 Comparación de tipos de datos:")
print("─" * 50)

comparison = pd.DataFrame({
    'Columna': df_pandas.columns,
    'Tipo Pandas': df_pandas.dtypes.astype(str).values,
    'Tipo Polars': [str(dt) for dt in df_polars.dtypes]
})

display(comparison)

---
## 6. Test 4: Escalabilidad

In [ ]:
print("="*60)
print("TEST 4: ESCALABILIDAD")
print("="*60)

test_sizes = [10_000, 100_000, 500_000, 1_000_000, 2_000_000]
scale_results = []

for size in test_sizes:
    print(f"\n📊 Probando {size:,} filas...")
    
    # Generar datos
    data = DataGenerator.create(size)
    test_pd = pd.DataFrame(data)
    test_pl = pl.DataFrame(data)
    
    # Benchmark: GroupBy
    r_pd = benchmark(
        lambda: test_pd.groupby(['category', 'region']).agg({'quantity': 'sum', 'unit_price': 'mean'}),
        warmup=1, runs=3
    )
    r_pl = benchmark(
        lambda: test_pl.group_by(['category', 'region']).agg([pl.col('quantity').sum(), pl.col('unit_price').mean()]),
        warmup=1, runs=3
    )
    
    speedup = r_pd['median'] / r_pl['median']
    scale_results.append({'Filas': size, 'Pandas': r_pd['median'], 'Polars': r_pl['median'], 'Speedup': speedup})
    
    print(f"   Pandas: {r_pd['median']*1000:.2f}ms | Polars: {r_pl['median']*1000:.2f}ms | Speedup: {speedup:.2f}x")
    
    del data, test_pd, test_pl
    gc.collect()

df_scale = pd.DataFrame(scale_results)

In [ ]:
# Visualización de escalabilidad
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Tiempos
ax1 = axes[0]
ax1.loglog(df_scale['Filas'], df_scale['Pandas'], 'o-', color=PANDAS_COLOR, 
           linewidth=2.5, markersize=10, label='Pandas', markeredgecolor='white', markeredgewidth=2)
ax1.loglog(df_scale['Filas'], df_scale['Polars'], 's-', color=POLARS_COLOR, 
           linewidth=2.5, markersize=10, label='Polars', markeredgecolor='white', markeredgewidth=2)
ax1.set_xlabel('Número de Filas', fontweight='bold')
ax1.set_ylabel('Tiempo (segundos)', fontweight='bold')
ax1.set_title('⏱️ Tiempo vs Tamaño del Dataset', fontweight='bold', fontsize=13)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')

# Gráfico 2: Speedup
ax2 = axes[1]
colors = [POLARS_COLOR if s > 1 else PANDAS_COLOR for s in df_scale['Speedup']]
bars = ax2.bar(range(len(df_scale)), df_scale['Speedup'], color=colors, alpha=0.8, edgecolor='white', linewidth=2)
ax2.axhline(y=1, color='red', linestyle='-', linewidth=2)

for i, (bar, val) in enumerate(zip(bars, df_scale['Speedup'])):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{val:.1f}x', ha='center', fontweight='bold')

ax2.set_xlabel('Número de Filas', fontweight='bold')
ax2.set_ylabel('Speedup', fontweight='bold')
ax2.set_title('🚀 Speedup de Polars', fontweight='bold', fontsize=13)
ax2.set_xticks(range(len(df_scale)))
ax2.set_xticklabels([f'{r:,}' for r in df_scale['Filas']], rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n⚡ Speedup promedio: {df_scale['Speedup'].mean():.2f}x")

---
## 7. Test 5: Compatibilidad y Ecosistema

In [ ]:
print("="*60)
print("TEST 5: COMPATIBILIDAD Y ECOSISTEMA")
print("="*60)

# Conversión entre formatos
print("\n🔄 Conversión entre Pandas y Polars:")
print("─" * 50)

# Polars -> Pandas
start = time.perf_counter()
converted_to_pandas = df_polars.to_pandas()
time_to_pandas = time.perf_counter() - start

# Pandas -> Polars
start = time.perf_counter()
converted_to_polars = pl.from_pandas(df_pandas)
time_to_polars = time.perf_counter() - start

print(f"   Polars → Pandas: {time_to_pandas*1000:.2f} ms")
print(f"   Pandas → Polars: {time_to_polars*1000:.2f} ms")
print("   ✅ Conversión bidireccional sencilla")

In [ ]:
# Lectura de archivos
print("\n📁 Lectura de archivos:")
print("─" * 50)

# Crear archivos temporales
import tempfile
import os

with tempfile.TemporaryDirectory() as tmpdir:
    csv_path = os.path.join(tmpdir, 'test.csv')
    parquet_path = os.path.join(tmpdir, 'test.parquet')
    
    # Guardar
    df_pandas.to_csv(csv_path, index=False)
    df_pandas.to_parquet(parquet_path, index=False)
    
    # CSV
    r_pd_csv = benchmark(lambda: pd.read_csv(csv_path), warmup=1, runs=3)
    r_pl_csv = benchmark(lambda: pl.read_csv(csv_path), warmup=1, runs=3)
    
    # Parquet
    r_pd_pq = benchmark(lambda: pd.read_parquet(parquet_path), warmup=1, runs=3)
    r_pl_pq = benchmark(lambda: pl.read_parquet(parquet_path), warmup=1, runs=3)

print(f"\n   CSV ({N_ROWS:,} filas):")
print(f"      Pandas: {r_pd_csv['median']*1000:.0f}ms | Polars: {r_pl_csv['median']*1000:.0f}ms | Speedup: {r_pd_csv['median']/r_pl_csv['median']:.2f}x")

print(f"\n   Parquet ({N_ROWS:,} filas):")
print(f"      Pandas: {r_pd_pq['median']*1000:.0f}ms | Polars: {r_pl_pq['median']*1000:.0f}ms | Speedup: {r_pd_pq['median']/r_pl_pq['median']:.2f}x")

In [ ]:
# Compatibilidad con NumPy y sklearn
print("\n🔗 Integración con NumPy:")
print("─" * 50)

# A NumPy
array_from_pandas = df_pandas[['quantity', 'unit_price']].values
array_from_polars = df_polars.select(['quantity', 'unit_price']).to_numpy()

print(f"   Pandas → NumPy: {array_from_pandas.shape}")
print(f"   Polars → NumPy: {array_from_polars.shape}")
print(f"   ✅ Ambos producen arrays compatibles")

# Con sklearn (si está disponible)
try:
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    scaled_pandas = scaler.fit_transform(array_from_pandas)
    scaled_polars = scaler.fit_transform(array_from_polars)
    
    print(f"\n🤖 Integración con sklearn:")
    print(f"   ✅ StandardScaler funciona con ambos")
except ImportError:
    print("\n   ⚠️ sklearn no instalado")

---
## 8. Comparación de Sintaxis

In [ ]:
print("="*60)
print("COMPARACIÓN DE SINTAXIS")
print("="*60)

print("""
┌──────────────────┬─────────────────────────────┬─────────────────────────────────┐
│ Operación        │ Pandas                      │ Polars                          │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Seleccionar      │ df[['a', 'b']]              │ df.select(['a', 'b'])           │
│ columnas         │                             │                                 │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Filtrar          │ df[df['a'] > 5]             │ df.filter(pl.col('a') > 5)      │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Nueva columna    │ df['c'] = df['a'] + 1       │ df.with_columns(                │
│                  │                             │   (pl.col('a') + 1).alias('c')) │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ GroupBy          │ df.groupby('a')['b'].sum()  │ df.group_by('a').agg(           │
│                  │                             │   pl.col('b').sum())            │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Ordenar          │ df.sort_values('a')         │ df.sort('a')                    │
├──────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Join             │ pd.merge(df1, df2, on='k')  │ df1.join(df2, on='k')           │
└──────────────────┴─────────────────────────────┴─────────────────────────────────┘

💡 Pandas: Sintaxis más concisa, estilo "pythónico"
💡 Polars: Sintaxis más explícita, mejor para queries complejos
""")

---
## 9. Conclusiones

In [ ]:
print("="*70)
print("📊 CONCLUSIONES DEL ANÁLISIS")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│                      RESULTADOS CLAVE                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ⚡ RENDIMIENTO                                                     │
│     • Polars: {df_results['Speedup'].mean():.1f}x más rápido en promedio                         │
│     • Ventaja en TODAS las operaciones testeadas                   │
│     • Mayor diferencia en GroupBy y Window Functions               │
│                                                                     │
│  💾 MEMORIA                                                         │
│     • Polars usa {((mem_pandas - mem_polars) / mem_pandas) * 100:.0f}% menos memoria                              │
│     • Tipos de datos más eficientes (Apache Arrow)                 │
│                                                                     │
│  📈 ESCALABILIDAD                                                   │
│     • Polars escala mejor con datasets grandes                     │
│     • Paralelización automática multi-core                         │
│     • Speedup consistente en entorno local: >{df_scale['Speedup'].min():.1f}x                     │
│                                                                     │
│  🔄 COMPATIBILIDAD                                                  │
│     • Conversión bidireccional fácil                               │
│     • Compatible con NumPy, sklearn, etc.                          │
│     • Soporta CSV, Parquet, JSON, etc.                             │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")

print("""
┌─────────────────────────────────────────────────────────────────────┐
│                    RECOMENDACIONES                                 │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  🐻‍❄️ USA POLARS:                                                    │
│     ✓ Pipelines de producción                                      │
│     ✓ Datasets grandes (>100K filas)                               │
│     ✓ Cuando el rendimiento importa                                │
│     ✓ ETL y procesamiento de datos                                 │
│                                                                     │
│  🐼 USA PANDAS:                                                     │
│     ✓ Prototipado rápido                                           │
│     ✓ Integración con librerías legacy                             │
│     ✓ Cuando tu equipo ya domina Pandas                            │
│     ✓ Documentación y ejemplos abundantes                          │
│                                                                     │
│  💡 ESTRATEGIA HÍBRIDA:                                             │
│     1. Prototipa con Pandas (más familiar)                         │
│     2. Migra a Polars para producción                              │
│     3. Usa .to_pandas() cuando necesites compatibilidad            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")

# Matriz de decisión
decision = pd.DataFrame({
    'Criterio': ['Rendimiento', 'Memoria', 'Escalabilidad', 'Facilidad de uso', 
                 'Ecosistema', 'Prototipado', 'Producción'],
    'Pandas': ['⭐⭐', '⭐⭐', '⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐'],
    'Polars': ['⭐⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐']
})

print("\n📋 MATRIZ DE DECISIÓN:")
display(decision)

In [ ]:
print(f"\n{'='*70}")
print(f" ✅ ANÁLISIS COMPLETADO - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*70}")